### Automated Event Planning

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from litellm import completion


ollama_llm = completion(
            model="ollama/llama3.2:latest",
            api_base="http://localhost:11434"
            )

In [7]:
from crewai import Agent, Crew, Process, Task
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

#### Creating Agents

In [8]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",

    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=ollama_llm,
    allow_delegation=True
)

In [9]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=ollama_llm,
    allow_delegation=True
)

In [10]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=ollama_llm,
    allow_delegation=False
)

### Creating Venue Pydantic Object
Agents will populate this object with information about different venues by creating different instances of it.

In [11]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [12]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator,
   
)

In [13]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    #async_execution=True,
    agent=logistics_manager,
  
)

In [14]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent,

)

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [15]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
           
    
    verbose=True
)

#### Define Input


In [16]:
event_details = {
    'event_topic': "Wedding Conference",
    'event_description': "My wedding is a unique event where I want to share my journey of love and commitment with my family and friends.",
    'event_city': "Yangon",
    'tentative_date': "2024-05-12",
    'expected_participants': 100,
    'budget': 1000000,
    'venue_type': "Venue"
}

In [ ]:

result = event_management_crew.kickoff(inputs=event_details)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: f274e40f-d2b2-4666-95c7-25e74c69de66                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 93b2e081-6686-4687-ae48-a0b4d57a5cc6
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 93b2e081-6686-4687-ae48-a0b4d57a5cc6
       Status: Executing Task...
    └── 🤖 Agent: Venue Coordinator
            Status: In Progress

# Agent: Venue Coordinator
## Task: Find a venue in Yangon that meets criteria for Wedding Conference.


🤖 Agent: Venue Coordinator
    Status: In Progress
└── 🧠 Thinking...

...

🤖 Agent: Venue Coordinator
    Status: ✅ Completed



# Agent: Logistics Manager
## Thought: Thought: The best course of action is to use the tool "Read website content" to gather more information about wedding venues.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.example.com/wedding-venues/\"}"
## Tool Output: 

Example Domain
Example Domain
This domain is for use in illustrative examples in documents. You may use this
 domain in literature without prior coordination or asking for permission.
More information...



🤖 Agent: Venue Coordinator
    Status: ✅ Completed
└── 🧠 Thinking...

🤖 Agent: Venue Coordinator
    Status: ✅ Completed



# Agent: Logistics Manager
## Thought: Thought: The best course of action is to use the tool "Ask question to coworker" to gather more information about wedding venues.
## Using tool: Ask question to coworker
## Tool Input: 
"{\"question\": \"What are some popular wedding venues?\", \"context\": \"Wedding venue options\", \"coworker\": \"Venue Coordinator\"}"
## Tool Output: 
I tried reusing the same input, I must stop using this action input. I'll try something else instead.




You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'
Tool Name: Read website content
Tool Arguments: {'website_url': {'description': 'Mandatory 

In [ ]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '', 'booking_status': '', 'capacity': 0, 'name': 'Human'}
